In [1]:
from scripts import helpers, mlogo

> Initiating required folders: 
> folder `input` exists, continuing.
> folder `output` exists, continuing.
> folder `output//motif_figure` exists, continuing.
> folder `output//motif_data`` exists, continuing.


In [ ]:
helpers.read_json()

In [ ]:
# Modules import
import os 
import pandas as pd

# Function that loads the sequences dataset as pandas dataframe.
def seq_import(seq_file:str) -> pd.DataFrame:
    
    # Cheecking if 'input' folder exists, if not -> create 'input' folder.
    # Defualt folder path in '/input', change config
    if os.path.exists:
        pass


In [ ]:
os.getcwd()

In [ ]:
from scripts.helpers import read_json

def create_folders():
    config_dict = read_json()
    main_folders = [config_dict["input_folder"], config_dict["output_folder"]]
    subf_output = [config_dict["output_folder"] + i for i in ["//motif_figure", "//motif_data`"]]

    for folder in main_folders + subf_output:
        if os.path.exists(folder) is False:
            os.mkdir(folder)
            print(f"> folder `{folder}` was created.")


create_folders()